In [46]:
import csv
from rdflib import Graph, URIRef, Literal, Namespace, RDF, XSD
from datetime import datetime
from rdflib.plugins.sparql import prepareQuery
import random
import random
import uuid

# Créez un espace de nom pour votre ontologie
kesaio = Namespace("http://www.semanticweb.org/mandiaye/ontologies/2023/7/CatastropheV2/")
common_core_ns = Namespace("http://www.ontologyrepository.com/CommonCoreOntologies/")
hasRole = URIRef("http://purl.obolibrary.org/obo/RO_0000087")
participatesIn = URIRef("http://purl.obolibrary.org/obo/RO_0000056")

In [47]:
# Ouvrez le fichier CSV d'entrée
with open("dataFromWikidata.csv", "r", newline="", encoding="utf-8") as infile:
    reader = csv.DictReader(infile)
    
    unique_records = {}
    
    for row in reader:
        row["seismeLabel"] = row["seismeLabel"].replace(" ", "_")
        row["lieuLabel"] = row["lieuLabel"].replace(" ", "_")
        row["paysLabel"] = row["paysLabel"].replace(" ", "_")
        row["description"] = row["description"].replace(" ", "_")

        key = (row["seismeLabel"])
        
        if key not in unique_records:
            unique_records[key] = row

# Créez un nouveau fichier CSV pour les enregistrements uniques
with open("dataFromWikidataNew.csv", "w", newline="", encoding="utf-8") as outfile:
    fieldnames = reader.fieldnames  # Utilisez les mêmes en-têtes de colonnes que le fichier d'origine
    writer = csv.DictWriter(outfile, fieldnames=fieldnames)
    
    writer.writeheader()
    
    for record in unique_records.values():
        writer.writerow(record)

In [48]:
def sévérité(kesaio, g, severite_niveau_0, severite_niveau_1, severite_niveau_2, seisme_uri, nombre_morts):
    if nombre_morts in severite_niveau_0:
        severite = "Niveau 0"
    elif nombre_morts in severite_niveau_1:
        severite = "Niveau 1"
    elif nombre_morts in severite_niveau_2:
        severite = "Niveau 2"
    else:
        severite = "Niveau inconnu"

        # Ajoutez une propriété de sévérité à l'instance de catastrophe
    g.add((seisme_uri, kesaio["sévérite_de_la_catastrophe"], Literal(severite, datatype=XSD.string)))

In [49]:
def creer_victimes(kesaio, g, dcd_uri, ua_uri, ur_uri, ump_uri, ind_uri, row, seisme_uri):
    l = [ua_uri, ur_uri, ump_uri, ind_uri]
    for i in range(0, int(row["nombreMorts"])): 
        victime_id = kesaio["victime_" + str(i)]  
        g.add((victime_id, RDF.type, kesaio["Victime"]))  
        g.add((victime_id, kesaio["has_status"], dcd_uri))
        g.add((seisme_uri, kesaio["engendre"], victime_id))


    for i in range(int(row["nombreMorts"]), int(row["nombreMorts"]) + int(row["nombreBlesse"])):
        element_aleatoire = random.choice(l)
        victime_id = kesaio["victime_" + str(i)]  
        g.add((victime_id, RDF.type, kesaio["Victime"]))  
        g.add((victime_id, kesaio["has_status"], element_aleatoire))
        g.add((seisme_uri, kesaio["engendre"], victime_id))

In [50]:
def catastrophe(kesaio, common_core_ns, g, severite_niveau_0, severite_niveau_1, severite_niveau_2, dcd_uri, ua_uri, ur_uri, ump_uri, ind_uri, creer_victimes, sévérité, row):
    date_uri = kesaio[row["date"]]
        
    # Ajoutez des triples RDF pour la classe Date
    g.add((date_uri, RDF.type, kesaio["Date"]))
        
    # Extraitz le jour, le mois et l'année du format de date "2016-02-06T00:00:00Z"
    date_str = row["date"]
    date_obj = datetime.strptime(date_str, "%Y-%m-%dT%H:%M:%SZ")
    jour = date_obj.day
    mois = date_obj.month
    annee = date_obj.year
        
    # Créez des ressources URI pour les composants jour, mois et année
    jour_uri = kesaio[f"jour_{jour}"]
    mois_uri = kesaio[f"mois_{mois}"]
    annee_uri = kesaio[f"{annee}"]

    # Ajoutez des triples RDF pour les composants jour, mois et année
    g.add((jour_uri, RDF.type, common_core_ns["GregorianDay"]))
    g.add((mois_uri, RDF.type, kesaio["Gregorian_Month"]))
    g.add((annee_uri, RDF.type, common_core_ns["GregorianYear"]))
        
    # Liez les composants jour, mois et année à l'instance de date
    g.add((date_uri, kesaio["has_day"], jour_uri))
    g.add((date_uri, kesaio["has_month"], mois_uri))
    g.add((date_uri, kesaio["has_year"], annee_uri))

    seisme_uri = kesaio[row["seismeLabel"]]
    pays_uri = kesaio[row["paysLabel"]]
    city_uri = kesaio[row["lieuLabel"]]

        
    # Ajoutez des triples RDF en utilisant l'espace de nom
    g.add((seisme_uri, RDF.type, kesaio["Séisme"]))
    g.add((pays_uri, RDF.type, common_core_ns["State"]))
    g.add((city_uri, RDF.type, common_core_ns["City"]))
        
    # Ajoutez des triples RDF pour les dataproperties
    g.add((seisme_uri, kesaio["nombre_de_morts"], Literal(row["nombreMorts"], datatype=XSD.integer)))
    g.add((seisme_uri, kesaio["nombre_de_blessés"], Literal(row["nombreBlesse"], datatype=XSD.integer)))
    g.add((seisme_uri, kesaio["description"], Literal(row["description"], datatype=XSD.string)))
        
    # Ajoutez des triples RDF pour les relations entre les instances
    g.add((seisme_uri, common_core_ns["occurs_at"], city_uri))
    g.add((city_uri, common_core_ns["spatial_part_of"], pays_uri))
    g.add((seisme_uri, common_core_ns["occurs_on"], date_uri))

    # Obtenez le nombre de morts à partir du CSV (assurez-vous que le champ est converti en entier)
    nombre_morts = int(row["nombreMorts"])
        
    # Déterminez la sévérité en fonction du nombre de morts
    sévérité(kesaio, g, severite_niveau_0, severite_niveau_1, severite_niveau_2, seisme_uri, nombre_morts)

    creer_victimes(kesaio, g, dcd_uri, ua_uri, ur_uri, ump_uri, ind_uri, row, seisme_uri)
    # Créez une instance unique de la classe "Sapeur_pompier" avec un identifiant unique (Sapeur_pompier_i)

    act_uri, uri_op = kesaio["Acte_de_réponse_"], kesaio["Opération_"]
    g.add((act_uri, RDF.type, kesaio["Acte_de_réponse"]))
    g.add((uri_op, RDF.type, kesaio["Opération"]))
    g.add((uri_op, kesaio["estSuperviséPAr"], kesaio["cos"]))
    g.add((act_uri, common_core_ns["has_process_part"], uri_op))
    g.add((seisme_uri, kesaio["trigger_to"], act_uri))

    pompier_id = kesaio["Commandant_des_opérations_de_secours"]
    identifiant_unique = uuid.uuid4()
    g.add((pompier_id, RDF.type, kesaio["Pompier"])) 
    g.add((pompier_id, hasRole, kesaio["cos"]))
    g.add((pompier_id, kesaio["identifiant"], Literal(identifiant_unique, datatype=XSD.string)))
    g.add((pompier_id, common_core_ns["is_affiliated_with"], kesaio["sapeurPompier"]))


    return uri_op


In [51]:
# Créez une instance de Graph pour votre ontologie RDF
g = Graph()

# Chargez votre ontologie RDF existante (si nécessaire)
g.parse("file9.rdf")

# Définissez les plages de nombre de morts pour chaque niveau de sévérité
severite_niveau_0 = range(0, 100)
severite_niveau_1 = range(100, 500)
severite_niveau_2 = range(500, 1000000)

dcd_uri = kesaio["décédé"]

ua_uri = kesaio["ua"]
ur_uri = kesaio["ur"]
# ud_uri = kesaio["ud"]
ump_uri = kesaio["ump"]
ind_uri = kesaio["indemne"]

with open("dataFromWikidataNew.csv", "r", newline="", encoding="utf-8") as csvfile:
    csvreader = csv.DictReader(csvfile)
    for row in csvreader:
        # Créez une ressource URI pour chaque instance de date
        uri_op = catastrophe(kesaio, common_core_ns, g, severite_niveau_0, severite_niveau_1, severite_niveau_2, dcd_uri, ua_uri, ur_uri, ump_uri, ind_uri, creer_victimes, sévérité, row)

# Questions de Compétences

In [52]:
# What is the nature of the disaster <X>?
requete_sparql = """
SELECT ?type ?description
WHERE {
    ?seisme rdf:type ?type.
    ?seisme kesaio:description ?description .
}
"""

# Préparez la requête SPARQL
requete = prepareQuery(requete_sparql, initNs={"kesaio": kesaio})

# Exécutez la requête et itérez sur les résultats
resultats = g.query(requete)

for resultat in resultats:
    # Obtenir le nom de la classe en extrayant la partie après le dernier "/"
    classe_nom = resultat.type.split("/")[-1]
    print("Nature:", classe_nom)
    print("Description:", resultat.description)

Nature: Séisme
Description: séisme_au_Japon_en_2018


In [53]:
# When did the disaster <X> take place?
requete_sparql = """
SELECT ?jour ?mois ?annee
WHERE {
    ?seisme cco:occurs_on ?date.
    ?date kesaio:has_day ?jour .
    ?date kesaio:has_month ?mois .
    ?date kesaio:has_year ?annee .
}
"""

# Préparez la requête SPARQL
requete = prepareQuery(requete_sparql, initNs={"kesaio": kesaio, "cco": common_core_ns})

# Exécutez la requête et itérez sur les résultats
resultats = g.query(requete)

for resultat in resultats:
    # Obtenez les valeurs en tant que chaînes de caractères
    jour = str(resultat.jour.split('/')[-1])
    mois = str(resultat.mois.split('/')[-1])
    annee = str(resultat.annee.split('/')[-1])

    print("Jour:", jour)
    print("Mois:", mois)
    print("Année:", annee)

Jour: jour_6
Mois: mois_9
Année: 2018


In [54]:
# Where did the disaster <X> take place?
requete_sparql = """
SELECT ?city ?pays
WHERE {
    ?seisme cco:occurs_at ?city.
    ?city cco:spatial_part_of ?pays
}
"""

# Préparez la requête SPARQL
requete = prepareQuery(requete_sparql, initNs={"cco": common_core_ns})

# Exécutez la requête et itérez sur les résultats
resultats = g.query(requete)

for resultat in resultats:
    # Obtenez les valeurs en tant que chaînes de caractères
    city = str(resultat.city.split('/')[-1])
    pays = str(resultat.pays.split('/')[-1])

    print("Ville:", city)
    print("Pays:", pays)

Ville: Tomakomai
Pays: Japon


In [55]:
# What is the criticality level of the disaster <X>?
requete_sparql = """
SELECT ?sévérité ?blessés ?morts
WHERE {
    ?seisme kesaio:nombre_de_morts ?morts .
    ?seisme kesaio:nombre_de_blessés ?blessés .
    ?seisme kesaio:sévérite_de_la_catastrophe ?sévérité .
}
"""

# Préparez la requête SPARQL
requete = prepareQuery(requete_sparql, initNs={"kesaio": kesaio})

# Exécutez la requête et itérez sur les résultats
resultats = g.query(requete)

for resultat in resultats:
    # Obtenez les valeurs en tant que chaînes de caractères
    sévérité = str(resultat.sévérité.split('/')[-1])
    blessés = str(resultat.blessés.split('/')[-1])
    morts = str(resultat.morts.split('/')[-1])

    print("Sévérité de la catastrophe:", sévérité)
    print("Nombre de blessés:", blessés)
    print("Nombre de morts:", morts)

Sévérité de la catastrophe: Niveau 0
Nombre de blessés: 10
Nombre de morts: 5


# PRV (Point de rassemblement des victimes)

In [56]:
coordonnees = kesaio[row["coordonnees"]]
# Trouvez l'indice du début des coordonnées
start_index = coordonnees.find("(")
# Trouvez l'indice de fin des coordonnées
end_index = coordonnees.find(")")
# Extrayez la partie des coordonnées entre les parenthèses
coord_part = coordonnees[start_index + 1:end_index]
# Séparez les coordonnées en latPRVitude et lonPRVgitude en utilisant l'espace comme séparateur
latPRV, lonPRV = coord_part.split()
# Convertissez les chaînes en nombres flottants si nécessaire
latPRV = float(latPRV) + 50
lonPRV = float(lonPRV) + 50

uri_prv = kesaio["Point_de_rassemblement_des_victimes" + "_1"]
g.add((uri_prv, RDF.type, kesaio["Point_de_rassemblement_des_victimes"]))
g.add((uri_prv, kesaio["longitude"], Literal(lonPRV, datatype=XSD.decimal)))
g.add((uri_prv, kesaio["latitude"], Literal(latPRV, datatype=XSD.decimal)))

uri_offPRV = kesaio["officierRassemblement"]

for i in range(0, 3): 
    identifiant_unique = uuid.uuid4()
    officierPRV_id = kesaio["OfficierPRV" + str(i)]  
    g.add((officierPRV_id, RDF.type, kesaio["Membre_SAMU"])) 
    g.add((officierPRV_id, common_core_ns["is_affiliated_with"], kesaio["samu"]))
    g.add((officierPRV_id, kesaio["identifiant"], Literal(identifiant_unique, datatype=XSD.string)))
    g.add((officierPRV_id, hasRole, uri_offPRV))
    g.add((uri_offPRV, kesaio["estSuperviséPAr"], kesaio["commandantSAMU"]))
    g.add((officierPRV_id, kesaio["monter"], uri_prv))
    # g.add((uri_prv, kesaio["installed_by"], uri_offPRV))

http://www.semanticweb.org/mandiaye/ontologies/2023/7/CatastropheV2/Point(84.725 28.165) does not look like a valid URI, trying to serialize this will break.


# Acte de Ramassage des victimes

In [57]:
uri_actDeRamassage = kesaio["Acte_de_ramassage_des_victimes" + "_1"]
g.add((uri_actDeRamassage, RDF.type, kesaio["Acte_de_ramassage_des_victimes"]))
g.add((uri_actDeRamassage, common_core_ns["occurs_at"], uri_prv))
g.add((uri_op, common_core_ns["has_process_part"], uri_actDeRamassage))

nombreMorts = int(row["nombreMorts"]) 
nombreBlesse = int(row["nombreBlesse"])

nombreDeVictime = nombreMorts + nombreBlesse

In [58]:
nombre_pompiers = 0
nombre_policiers = 0
nombre_officiers_samu = 0

if nombreDeVictime < 8:
    nombre_pompiers = 1
    nombre_policiers = 1
    nombre_officiers_samu = 1
elif 8 <= nombreDeVictime <= 25:
    nombre_pompiers = 5
    nombre_policiers = 2
    nombre_officiers_samu = 5
else:
    nombre_pompiers = 10
    nombre_policiers = 5
    nombre_officiers_samu = 10

In [59]:
for i in range(0,nombre_pompiers): 
    pompier_id = kesaio["Pompier" + str(i)]
    identifiant_unique = uuid.uuid4()
    g.add((pompier_id, RDF.type, kesaio["Pompier"])) 
    g.add((pompier_id, hasRole, kesaio["equipierMedical"]))
    g.add((pompier_id, kesaio["identifiant"], Literal(identifiant_unique, datatype=XSD.string)))
    g.add((pompier_id, common_core_ns["is_affiliated_with"], kesaio["sapeurPompier"]))
    g.add((pompier_id, participatesIn, uri_actDeRamassage))
for i in range(0,nombre_policiers): 
    # Instanciation des policiers
    policier_id = kesaio["Policier" + str(i)]
    identifiant_unique = uuid.uuid4()
    g.add((policier_id, RDF.type, kesaio["Policier"]))
    g.add((policier_id, hasRole, kesaio["encadrementEtapplication"]))  # Assurez-vous que le rôle est correct.
    g.add((policier_id, kesaio["identifiant"], Literal(identifiant_unique, datatype=XSD.string)))
    g.add((policier_id, common_core_ns["is_affiliated_with"], kesaio["policeNationale"]))
    g.add((policier_id, participatesIn, uri_actDeRamassage))
for i in range(0,nombre_officiers_samu): 
    # Instanciation des officiers du SAMU
    officier_samu_id = kesaio["OfficierRamassage" + str(i)]
    identifiant_unique = uuid.uuid4()
    g.add((officier_samu_id, RDF.type, kesaio["Membre_SAMU"]))
    g.add((officier_samu_id, hasRole, uri_offPRV))  # Assurez-vous que le rôle est correct.
    g.add((officier_samu_id, kesaio["identifiant"], Literal(identifiant_unique, datatype=XSD.string)))
    g.add((officier_samu_id, common_core_ns["is_affiliated_with"], kesaio["samu"]))
    g.add((officier_samu_id, participatesIn, uri_actDeRamassage))

In [60]:
# Écrire et exécuter une requête SPARQL pour obtenir toutes les instances de la classe "Victime"
query = """
    SELECT ?victime
    WHERE {
        ?victime rdf:type :Victime.
    }
"""
results = g.query(query)
# Parcourir les résultats et obtenir les instances de la classe "Victime"
for row in results:
    victime_instance = row.victime
    g.add((uri_actDeRamassage, kesaio["ramasser"], victime_instance))
    g.add((victime_instance, kesaio["ramassé"], Literal(True, datatype=XSD.boolean)))

# Le poste médical avancé (PMA)

In [61]:
latPMA = float(latPRV) + 1000
lonPMA = float(lonPRV) + 1000

uri_pma = kesaio["Poste_médical_avancé" + "_1"]
g.add((uri_pma, RDF.type, kesaio["Poste_médical_avancé"]))
g.add((uri_pma, kesaio["longitude"], Literal(lonPMA, datatype=XSD.decimal)))
g.add((uri_pma, kesaio["latitude"], Literal(latPMA, datatype=XSD.decimal)))
g.add((uri_pma, kesaio["nombreDeLit"], Literal(50, datatype=XSD.integer)))

uri_rolePMA = kesaio["officierPMA"]

for i in range(0, 3): 
    identifiant_unique = uuid.uuid4()
    uri_OPMA = kesaio["OfficierPMA" + str(i)]  
    g.add((uri_OPMA, RDF.type, kesaio["Membre_SAMU"])) 
    g.add((uri_OPMA, common_core_ns["is_affiliated_with"], kesaio["samu"]))
    g.add((uri_OPMA, kesaio["identifiant"], Literal(identifiant_unique, datatype=XSD.string)))
    g.add((uri_OPMA, hasRole, uri_rolePMA))
    g.add((uri_rolePMA, kesaio["estSuperviséPAr"], kesaio["commandantSAMU"]))
    g.add((uri_OPMA, kesaio["monter"], uri_pma))
    # g.add((uri_pma, kesaio["installed_by"], uri_rolePMA))

### Acte de secours des victimes

In [62]:
uri_actDeSoins = kesaio["Acte_de_secours_des_victimes" + "_1"]
g.add((uri_actDeSoins, RDF.type, kesaio["Acte_de_secours_des_victimes"]))
g.add((uri_actDeSoins, common_core_ns["occurs_at"], uri_pma))
g.add((uri_op, common_core_ns["has_process_part"], uri_actDeSoins))
g.add((uri_actDeSoins, kesaio["needs"], kesaio["equipeMédicale"]))
g.add((uri_actDeSoins, kesaio["needs"], kesaio["vehiculeDeSecoursEtDassistanceAuxVictimes"]))
g.add((uri_actDeSoins, kesaio["needs"], kesaio["vehiculeDeSoutienSanitaire"]))

nombreBlesse

10

In [63]:
requete_sparql = """
    SELECT (COUNT(?victime) as ?blessesASoinger) 
    WHERE {
        ?victime rdf:type kesaio:Victime.
        ?victime kesaio:has_status ?Code_etat.
        FILTER (?Code_etat = kesaio:ur)
    }
"""

# Préparez la requête SPARQL
requete = prepareQuery(requete_sparql, initNs={"kesaio": kesaio})

results = g.query(requete)

# Parcourir les résultats et afficher les victimes
for row in results:
    blessesASoinger = row.blessesASoinger

blessesASoinger = int(blessesASoinger)
print(blessesASoinger)

4


In [64]:
# Calculez combien de médecins et d'infirmiers sont nécessaires
nombre_de_medecins = blessesASoinger // 5
nombre_d_infirmiers = blessesASoinger // 5

# Appliquez la règle
if blessesASoinger % 5 != 0:
    nombre_de_medecins += 1
    nombre_d_infirmiers += 1

# Affichez les résultats
print("Nombre total de victimes :", blessesASoinger)
print("Nombre de médecins nécessaires :", nombre_de_medecins)
print("Nombre d'infirmiers nécessaires :", nombre_d_infirmiers)

Nombre total de victimes : 4
Nombre de médecins nécessaires : 1
Nombre d'infirmiers nécessaires : 1


In [65]:
for i in range(0,nombre_de_medecins): 
    # Instanciation des officiers du SAMU
    medecin = kesaio["Médecin_urgentiste_" + str(i)]
    identifiant_unique = uuid.uuid4()
    g.add((medecin, RDF.type, kesaio["Membre_SAMU"]))
    g.add((medecin, hasRole, uri_offPRV))  # Assurez-vous que le rôle est correct.
    g.add((medecin, kesaio["identifiant"], Literal(identifiant_unique, datatype=XSD.string)))
    g.add((medecin, common_core_ns["is_affiliated_with"], kesaio["samu"]))
    g.add((medecin, participatesIn, uri_actDeSoins))

for i in range(0,nombre_d_infirmiers): 
    # Instanciation des officiers du SAMU
    infirmier = kesaio["Infirmier_SAMU_" + str(i)]
    identifiant_unique = uuid.uuid4()
    g.add((infirmier, RDF.type, kesaio["Membre_SAMU"]))
    g.add((infirmier, hasRole, uri_offPRV))  # Assurez-vous que le rôle est correct.
    g.add((infirmier, kesaio["identifiant"], Literal(identifiant_unique, datatype=XSD.string)))
    g.add((infirmier, common_core_ns["is_affiliated_with"], kesaio["samu"]))
    g.add((infirmier, participatesIn, uri_actDeSoins))

In [66]:
query = """
    SELECT ?victime
    WHERE {
        ?victime rdf:type kesaio:Victime.
        ?victime kesaio:has_status ?Code_etat.
        FILTER (?Code_etat = kesaio:ur)
    }
"""
# Préparez la requête SPARQL
requete = prepareQuery(query, initNs={"kesaio": kesaio})

results = g.query(requete)
# Parcourir les résultats et obtenir les instances de la classe "Victime"
for row in results:
    victime_instance = row.victime
    g.add((uri_actDeSoins, kesaio["soigner"], victime_instance))
    g.add((victime_instance, kesaio["soigné"], Literal(True, datatype=XSD.boolean)))

##### Suivi psycho medical

In [67]:
requete_sparql = """
    SELECT (COUNT(?victime) as ?suivi_psycho_medical) 
    WHERE {
        ?victime rdf:type kesaio:Victime.
        ?victime kesaio:has_status ?Code_etat.
        FILTER (?Code_etat = kesaio:ump)
    }
"""

# Préparez la requête SPARQL
requete = prepareQuery(requete_sparql, initNs={"kesaio": kesaio})

results = g.query(requete)

# Parcourir les résultats et afficher les victimes
for row in results:
    suivi_psycho_medical = row.suivi_psycho_medical

suivi_psycho_medical = int(suivi_psycho_medical)
print(suivi_psycho_medical)

2


In [68]:
nombre_psychologues = suivi_psycho_medical // 5

if suivi_psycho_medical % 5 != 0:
    nombre_psychologues += 1

for i in range(0,nombre_psychologues): 
    # Instanciation des officiers du SAMU
    infirmier = kesaio["Psychologue_SAMU_" + str(i)]
    identifiant_unique = uuid.uuid4()
    g.add((infirmier, RDF.type, kesaio["Membre_SAMU"]))
    g.add((infirmier, hasRole, uri_offPRV))  # Assurez-vous que le rôle est correct.
    g.add((infirmier, kesaio["identifiant"], Literal(identifiant_unique, datatype=XSD.string)))
    g.add((infirmier, common_core_ns["is_affiliated_with"], kesaio["samu"]))
    g.add((infirmier, participatesIn, uri_actDeSoins))

query = """
    SELECT ?victime
    WHERE {
        ?victime rdf:type kesaio:Victime.
        ?victime kesaio:has_status ?Code_etat.
        FILTER (?Code_etat = kesaio:ump)
    }
"""
# Préparez la requête SPARQL
requete = prepareQuery(query, initNs={"kesaio": kesaio})

results = g.query(requete)
# Parcourir les résultats et obtenir les instances de la classe "Victime"
for row in results:
    victime_instance = row.victime
    g.add((uri_actDeSoins, kesaio["soigner"], victime_instance))
    g.add((victime_instance, kesaio["soigné"], Literal(True, datatype=XSD.boolean)))

##### Gestion des morts

In [69]:
nombre_pompiers = nombreMorts // 5

for i in range(0,nombre_pompiers): 
    pompier_id = kesaio["Pompier" + str(i)]
    identifiant_unique = uuid.uuid4()
    g.add((pompier_id, RDF.type, kesaio["Pompier"])) 
    g.add((pompier_id, hasRole, kesaio["equipierMedical"]))
    g.add((pompier_id, kesaio["identifiant"], Literal(identifiant_unique, datatype=XSD.string)))
    g.add((pompier_id, common_core_ns["is_affiliated_with"], kesaio["sapeurPompier"]))
    g.add((pompier_id, participatesIn, uri_actDeSoins))


query = """
    SELECT ?victime
    WHERE {
        ?victime rdf:type kesaio:Victime.
        ?victime kesaio:has_status ?Code_etat.
        FILTER (?Code_etat = kesaio:décédé)
    }
"""
# Préparez la requête SPARQL
requete = prepareQuery(query, initNs={"kesaio": kesaio})

results = g.query(requete)
# Parcourir les résultats et obtenir les instances de la classe "Victime"
for row in results:
    victime_instance = row.victime
    g.add((victime_instance, kesaio["deposerAlaMorgue"], Literal(True, datatype=XSD.boolean)))

##### Gestion des rescapés

In [70]:
requete_sparql = """
    SELECT ?victime
    WHERE {
        ?victime rdf:type kesaio:Victime.
        ?victime kesaio:has_status ?Code_etat.
        FILTER (?Code_etat = kesaio:indemne)
    }
"""

# Préparez la requête SPARQL
requete = prepareQuery(requete_sparql, initNs={"kesaio": kesaio})

# Parcourir les résultats et afficher les victimes
results = g.query(requete)
# Parcourir les résultats et obtenir les instances de la classe "Victime"
for row in results:
    victime_instance = row.victime
    g.add((victime_instance, kesaio["relogé"], Literal(True, datatype=XSD.boolean)))

### Acte d'évacuation des victimes

In [71]:
requete_sparql = """
    SELECT (COUNT(?victime) as ?blessesAEvacuer) 
    WHERE {
        ?victime rdf:type kesaio:Victime.
        ?victime kesaio:has_status ?Code_etat.
        FILTER (?Code_etat = kesaio:ua)
    }
"""

# Préparez la requête SPARQL
requete = prepareQuery(requete_sparql, initNs={"kesaio": kesaio})

results = g.query(requete)

# Parcourir les résultats et afficher les victimes
for row in results:
    blessesAEvacuer = row.blessesAEvacuer

blessesAEvacuer = int(blessesAEvacuer)
print(blessesAEvacuer)

3


In [72]:
uri_actDEvacution = kesaio["Acte_devacuation_des_victimes" + "_1"]
g.add((uri_actDEvacution, RDF.type, kesaio["Acte_devacuation_des_victimes"]))
g.add((uri_actDEvacution, common_core_ns["occurs_at"], uri_pma))
g.add((uri_op, common_core_ns["has_process_part"], uri_actDEvacution))
g.add((uri_actDEvacution, kesaio["needs"], kesaio["equipeMédicale"]))

<Graph identifier=Nb85e2280e9704af091fdb3d9ab40fa85 (<class 'rdflib.graph.Graph'>)>

In [73]:
nombre_ambulance = blessesAEvacuer // 5
nombre_helicoptere = blessesAEvacuer // 10

if blessesASoinger % 5 != 0:
    nombre_ambulance += 1

for i in range(0,nombre_ambulance): 
    ambulance_id = kesaio["Ambulance_" + str(i)]
    g.add((ambulance_id, RDF.type, kesaio["Ambulance"])) 
    g.add((ambulance_id, kesaio["has_status"], kesaio["engagé"]))
    g.add((uri_actDEvacution, kesaio["needs"], ambulance_id))

    ambulancier = kesaio["ambulancier_" + str(i)]
    identifiant_unique = uuid.uuid4()
    g.add((ambulancier, RDF.type, kesaio["Membre_SAMU"]))
    g.add((ambulancier, hasRole, kesaio["ambulancier"]))  
    g.add((ambulancier, kesaio["identifiant"], Literal(identifiant_unique, datatype=XSD.string)))
    g.add((ambulancier, common_core_ns["is_affiliated_with"], kesaio["samu"]))
    g.add((ambulancier, kesaio["is_required"], uri_actDEvacution))
    g.add((ambulancier, participatesIn, uri_actDEvacution))

In [74]:
for i in range(0,nombre_helicoptere): 
    helico_id = kesaio["helicoptereSAMU_" + str(i)]
    g.add((helico_id, RDF.type, kesaio["Helicoptere_SAMU"])) 
    g.add((helico_id, kesaio["has_status"], kesaio["engagé"]))
    g.add((uri_actDEvacution, kesaio["needs"], helico_id))

    pilote = kesaio["pilote_" + str(i)]
    identifiant_unique = uuid.uuid4()
    g.add((pilote, RDF.type, kesaio["Membre_SAMU"]))
    g.add((pilote, hasRole, kesaio["pilote"]))  
    g.add((pilote, kesaio["identifiant"], Literal(identifiant_unique, datatype=XSD.string)))
    g.add((pilote, common_core_ns["is_affiliated_with"], kesaio["samu"]))
    g.add((pilote, kesaio["is_required"], uri_actDEvacution))
    g.add((pilote, participatesIn, uri_actDEvacution))

In [75]:
requete_sparql = """
    SELECT ?victime
    WHERE {
        ?victime rdf:type kesaio:Victime.
        ?victime kesaio:has_status ?Code_etat.
        FILTER (?Code_etat = kesaio:ua)
    }
"""

# Préparez la requête SPARQL
requete = prepareQuery(requete_sparql, initNs={"kesaio": kesaio})

# Parcourir les résultats et afficher les victimes
results = g.query(requete)
# Parcourir les résultats et obtenir les instances de la classe "Victime"
for row in results:
    victime_instance = row.victime
    g.add((uri_actDEvacution, kesaio["évacuer"], victime_instance))
    g.add((victime_instance, kesaio["évacué"], Literal(True, datatype=XSD.boolean)))

In [76]:
# owlrl.DeductiveClosure(owlrl.RDFS_Semantics, datatype_axioms=True).expand(g)
# owlrl.DeductiveClosure(owlrl.OWLRL_Extension, datatype_axioms=True).expand(g)

# Questions de Compétences

In [77]:
# Which ERs are involved in the operation <X>?
query = """
    SELECT DISTINCT ?secouriste ?organisation ?role ?identifiant
    WHERE {
        ?x rdf:type/rdfs:subClassOf* kesaio:Catastrophe.
        ?x kesaio:trigger_to ?y.
        ?y cco:has_process_part ?z.
        ?z cco:has_process_part ?w.
        ?secouriste <http://purl.obolibrary.org/obo/RO_0000056> ?w.
        ?secouriste kesaio:identifiant ?identifiant.
        ?secouriste cco:is_affiliated_with ?organisation.
        ?secouriste <http://purl.obolibrary.org/obo/RO_0000087> ?role
    }
"""

# Préparer la requête SPARQL
requete = prepareQuery(query, initNs={"kesaio": kesaio, "cco": common_core_ns})

# Exécuter la requête SPARQL
results = g.query(requete)


resultats_par_organisation = {}

for row in results:
    secouriste = row["secouriste"]
    organisation = row["organisation"]
    role = row["role"]
    identifiant = row["identifiant"]

    secouriste_nom = secouriste.split("/")[-1]  
    organisation_nom = organisation.split("/")[-1]  
    role_nom = role.split("/")[-1]  
    

    if organisation_nom not in resultats_par_organisation:
        resultats_par_organisation[organisation_nom] = []


    resultats_par_organisation[organisation_nom].append((secouriste_nom, role_nom, identifiant))

# Afficher les résultats regroupés
for organisation, secouristes in resultats_par_organisation.items():
    print("Organisation:", organisation)
    for secouriste_info in secouristes:
        secouriste, role, identifiant = secouriste_info
        print("  Secouriste:", secouriste)
        print("  Rôle:", role)
        print("  Identifiant:", identifiant)


Organisation: sapeurPompier
  Secouriste: Pompier0
  Rôle: equipierMedical
  Identifiant: 0a8007b9-d4ec-4ea2-83cb-bd63d3b6a351
  Secouriste: Pompier0
  Rôle: equipierMedical
  Identifiant: 637f62ef-4a3e-48e9-ac92-dfb2625ebfa2
  Secouriste: Pompier1
  Rôle: equipierMedical
  Identifiant: 71f901d1-0928-42f0-bfae-03ccadad7390
  Secouriste: Pompier2
  Rôle: equipierMedical
  Identifiant: 41b05bdb-ad77-461e-89f4-695a73b425e7
  Secouriste: Pompier3
  Rôle: equipierMedical
  Identifiant: dc09fdfa-81a5-4b6b-9f51-c501aa5c61f8
  Secouriste: Pompier4
  Rôle: equipierMedical
  Identifiant: 0c296d2b-3f44-4bdb-9dc8-f511dbdaa30f
Organisation: policeNationale
  Secouriste: Policier0
  Rôle: encadrementEtapplication
  Identifiant: 9b80ed13-7144-4bfa-86b1-cd75b1dd5dce
  Secouriste: Policier1
  Rôle: encadrementEtapplication
  Identifiant: 31214887-45c5-42c3-8c78-f1535ea6f46e
Organisation: samu
  Secouriste: OfficierRamassage0
  Rôle: officierRassemblement
  Identifiant: 8aeccc4b-833c-4022-a1d8-bb4a178b8

In [78]:
# Where was the advanced medical post of <X>’s healthcare units located?
query = """
SELECT ?posteMedical ?latitude ?longitude
WHERE {
    ?posteMedical a kesaio:Poste_médical_avancé.
    ?posteMedical kesaio:latitude ?latitude.
    ?posteMedical kesaio:longitude ?longitude.
}
"""

# Préparez la requête SPARQL
requete = prepareQuery(query, initNs={"kesaio": kesaio})

# Exécutez la requête SPARQL et affichez les résultats
results = g.query(requete)

for row in results:
    poste_medical = row["posteMedical"].split("/")[-1]  
    latitude = row["latitude"]
    longitude = row["longitude"]

    print("Poste médical avancé :", poste_medical)
    print("Latitude :", latitude)
    print("Longitude :", longitude)
    print("\n")

Poste médical avancé : Poste_médical_avancé_1
Latitude : 1134.725
Longitude : 1078.165




In [79]:
# Who was the operational commander of the <X> operation?
query = """
SELECT ?roleCommandant ?commandant ?typeCommandant
WHERE {
    kesaio:Opération_ kesaio:estSuperviséPAr ?roleCommandant.
    ?commandant <http://purl.obolibrary.org/obo/RO_0000087> ?roleCommandant .
    ?commandant rdf:type ?typeCommandant.

}
"""

# Préparez la requête SPARQL
requete = prepareQuery(query, initNs={"kesaio": kesaio})

# Exécutez la requête SPARQL pour obtenir le commandant de l'opération
results = g.query(requete)

# Affichez le résultat
for row in results: 
    role_commandant = row["roleCommandant"].split("/")[-1]
    commandant = row["commandant"].split("/")[-1] 
    type_commandant = row["typeCommandant"].split("/")[-1] 

    print("Commandant :", commandant)
    print("Rôle :", role_commandant)
    print("Type :", type_commandant)

Commandant : Commandant_des_opérations_de_secours
Rôle : cos
Type : Pompier


In [80]:
# Who was the public authority that commanded the <X> operation?

query = """
SELECT ?role ?autorité 
WHERE {
    kesaio:Opération_ kesaio:estSuperviséPAr ?roleCommandant.
    ?roleCommandant kesaio:estSuperviséPAr ?dsi.
    ?dsi kesaio:estSuperviséPAr ?dsis.
    ?role kesaio:supervise ?dsis.
    ?autorité <http://purl.obolibrary.org/obo/RO_0000087> ?role .
}

"""

# Préparez la requête SPARQL
requete = prepareQuery(query, initNs={"kesaio": kesaio})

# Exécutez la requête SPARQL pour obtenir le commandant de l'opération
results = g.query(requete)

for row in results:
    role_autorite = row["role"].split("/")[-1]
    autorite = row["autorité"].split("/")[-1]

    print("Autorité publique :", autorite)
    print("Rôle d'autorité :", role_autorite)

Autorité publique : a5
Rôle d'autorité : ministre_de_l'interieur


In [81]:
# What were the actions of the ER [Y] in the operation <X>?

requete_sparql = """
SELECT DISTINCT ?action
WHERE {
    ?x rdf:type/rdfs:subClassOf* cco:OrganizationMember.
    ?x <http://purl.obolibrary.org/obo/RO_0000056> ?action.
}
"""

# Préparez la requête SPARQL
requete = prepareQuery(requete_sparql, initNs={"kesaio": kesaio, "cco": common_core_ns})

# Exécutez la requête et itérez sur les résultats
resultats = g.query(requete)

for row in resultats:
    actions = row["action"].split("/")[-1]

    print("Actions des ERs :", actions)

Actions des ERs : Acte_de_ramassage_des_victimes_1
Actions des ERs : Acte_de_secours_des_victimes_1
Actions des ERs : Acte_devacuation_des_victimes_1


In [82]:
# What is the definition of the term absolute emergency?
query = """
SELECT ?comment
WHERE {
    {kesaio:UA rdfs:comment ?comment.}
    UNION
    {kesaio:P0 rdfs:comment ?comment.}
}
"""

# Préparez la requête SPARQL
requete = prepareQuery(query, initNs={"kesaio": kesaio})

# Exécutez la requête SPARQL pour récupérer le commentaire
results = g.query(requete)

for row in results:
    comment = row["comment"]
    print("Commentaire :", comment)


Commentaire : Urgence Absolue
Commentaire : Déclenchement SMUR et ensuite régulation priorité
SMUR = structure mobile d’urgence et de réanimation


In [83]:
# Who is competent to search and rescue persons drowning as a result of <X>?
query = """
SELECT ?equipePlongee
WHERE {
    ?equipePlongee rdf:type/rdfs:subClassOf* kesaio:Equipe_de_plongée.
}
"""

# Préparez la requête SPARQL
requete = prepareQuery(query, initNs={"kesaio": kesaio})

# Exécutez la requête SPARQL pour récupérer le commentaire
results = g.query(requete)

for row in results:
    equipePlongee = row["equipePlongee"].split("/")[-1]
    print("Equipe :", equipePlongee)


Equipe : equipePlongée


In [84]:
# How many people were affected by <X>?

query = """
SELECT (COUNT(DISTINCT ?victime) AS ?nombreVictimes) (COUNT(DISTINCT ?secouriste) AS ?nombreSecouristes)
WHERE {
    ?victime rdf:type/rdfs:subClassOf* kesaio:Victime.
    ?secouriste rdf:type/rdfs:subClassOf* cco:OrganizationMember.
    ?secouriste <http://purl.obolibrary.org/obo/RO_0000056> ?action.
}
"""

# Préparez la requête SPARQL
requete = prepareQuery(query, initNs={"kesaio": kesaio, "cco": common_core_ns})

# Exécutez la requête SPARQL pour obtenir le nombre de victimes et de secouristes
results = g.query(requete)

# Affichez les résultats
for row in results:
    nombre_victimes = row["nombreVictimes"]
    nombre_secouristes = row["nombreSecouristes"]
    
    print("Nombre de victimes :", nombre_victimes)
    print("Nombre de secouristes :", nombre_secouristes)
    print("Nombre de personnes affectées :", nombre_victimes + nombre_secouristes)

Nombre de victimes : 15
Nombre de secouristes : 16
Nombre de personnes affectées : 31


In [85]:
# What was the state of the victim [W] of <X>?

numero_victime = random.randint(0, int(nombre_victimes)) 

# Définir la requête SPARQL avec le numéro aléatoire
query = f"""
SELECT ?etatVictime
WHERE {{
    kesaio:victime_{numero_victime} kesaio:has_status ?etatVictime.
}}
"""

# Préparez la requête SPARQL
requete = prepareQuery(query, initNs={"kesaio": kesaio})

# Exécutez la requête SPARQL pour obtenir l'état de la victime correspondante
results = g.query(requete)

# Affichez le résultat
for row in results:
    etat_victime = row["etatVictime"].split('/')[-1]
    print(f"État de la victime_{numero_victime} : {etat_victime}")

In [86]:
# What means of transport were used in the operation <X>?

requete_sparql = """
SELECT DISTINCT ?type 
WHERE {
    ?x rdf:type/rdfs:subClassOf* cco:OrganizationMember.
    ?x <http://purl.obolibrary.org/obo/RO_0000056> ?action.
    ?action kesaio:needs ?means.
    ?means rdf:type ?type .
    FILTER (?type != owl:NamedIndividual)
    FILTER (?type != kesaio:Equipe_médicale) 
}
"""

# Préparez la requête SPARQL
requete = prepareQuery(requete_sparql, initNs={"kesaio": kesaio, "cco": common_core_ns})

# Exécutez la requête et itérez sur les résultats
resultats = g.query(requete)

for row in resultats:
    actions = row["type"].split("/")[-1]

    print("Moyens de transports des ERs :", actions)


Moyens de transports des ERs : Véhicule_de_secours_et_d'assistance_aux_victimes
Moyens de transports des ERs : Véhicule_de_soutien_sanitaire
Moyens de transports des ERs : Ambulance


In [87]:
# What types of means are needed to respond to a forest fire?


# Définissez la requête SPARQL pour rechercher les types de moyens nécessaires pour répondre à un incendie de forêt
requete_sparql = """
SELECT DISTINCT ?objetDeBesoin
WHERE {
    kesaio:Acte_dextinction_dincendie rdfs:subClassOf* ?restriction.
    ?restriction owl:onProperty kesaio:needs.
    ?restriction owl:someValuesFrom ?objetDeBesoin.
}
"""

# Préparez la requête SPARQL
requete = prepareQuery(requete_sparql, initNs={"kesaio": kesaio})

# Exécutez la requête et itérez sur les résultats
resultats = g.query(requete)

# Affichez les résultats
for row in resultats:
    means = row["objetDeBesoin"].split("/")[-1]
    print("Types de moyens nécessaires pour répondre à un incendie de forêt :", means)


Types de moyens nécessaires pour répondre à un incendie de forêt : Camion_de_citerne_feux_de_foret
Types de moyens nécessaires pour répondre à un incendie de forêt : Fourgon_pompe_tonne


In [88]:
# Enregistrez l'ontologie enrichie dans un fichier RDF (si nécessaire)
g.serialize("ontologie_enrichie.rdf", format="xml")

for triple in g:
    print(triple)

(rdflib.term.URIRef('http://www.semanticweb.org/mandiaye/ontologies/2023/7/CatastropheV2/Pompier1'), rdflib.term.URIRef('http://www.semanticweb.org/mandiaye/ontologies/2023/7/CatastropheV2/identifiant'), rdflib.term.Literal('71f901d1-0928-42f0-bfae-03ccadad7390', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')))
(rdflib.term.URIRef('http://www.semanticweb.org/mandiaye/ontologies/2023/7/CatastropheV2/Ministre_des_Solidarités_et_de_la_Santé'), rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#subClassOf'), rdflib.term.URIRef('http://www.semanticweb.org/mandiaye/ontologies/2023/7/CatastropheV2/Rôle_Autorité_Publique'))
(rdflib.term.URIRef('http://www.semanticweb.org/mandiaye/ontologies/2023/7/CatastropheV2/maire'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://www.semanticweb.org/mandiaye/ontologies/2023/7/CatastropheV2/Maire'))
(rdflib.term.URIRef("http://www.semanticweb.org/mandiaye/ontologies/2023/7/Catas

In [89]:
# SELECT DISTINCT ?seismeLabel ?lieuLabel ?paysLabel ?date ?nombreMorts ?nombreBlesse ?description ?magnitude ?coordonnees
# WHERE {
#   ?seisme wdt:P31 wd:Q7944.
#   {?seisme wdt:P276 ?lieu.} UNION {?seisme wdt:P131 ?lieu.} UNION {?seisme wdt:P706 ?lieu.}
#   ?seisme wdt:P17 ?pays.
#   ?seisme wdt:P585 ?date. 
#   ?seisme schema:description ?description . FILTER(LANG(?description) = "fr") 
#   ?seisme wdt:P1120 ?nombreMorts.
#   ?seisme wdt:P1339 ?nombreBlesse .
#   ?seisme wdt:P2528 ?magnitude.
#   ?seisme wdt:P625 ?coordonnees.
#   SERVICE wikibase:label { bd:serviceParam wikibase:language "fr". }
# }


In [90]:
# SELECT DISTINCT ?disasterLabel ?lieuLabel ?paysLabel ?date ?nombreMorts ?nombreBlesse ?description ?magnitude ?coordonnees
# WHERE {
#   ?disaster wdt:P31 wd:Q7944.
#   ?disaster wdt:P131 ?lieu.
#   ?disaster wdt:P17 ?pays.
#   ?disaster wdt:P585 ?date. 
#   ?disaster schema:description ?description . FILTER(LANG(?description) = "fr") 
#   ?disaster wdt:P1120 ?nombreMorts.
#   ?disaster wdt:P1339 ?nombreBlesse .
#   OPTIONAL {?disaster wdt:P2528 ?magnitude.}
#   ?disaster wdt:P625 ?coordonnees.
#   SERVICE wikibase:label { bd:serviceParam wikibase:language "fr". }
# }